**JOIN's SQL training session - Poludniowy Albanczyk**
---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [3]:
tables= list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables]# Your code here

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [4]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,68472973,None,<p>Thank you. After thinking through your ans...,None,None,0,NaT,2021-07-21 16:09:49.560000+00:00,None,2021-07-21 16:09:49.560000+00:00,NaT,None,NaN,None,11880888,68461807,2,0,None,None
1,68472974,None,<pre><code>SELECT age((((m.message -&gt; 'head...,None,None,0,NaT,2021-07-21 16:09:54.130000+00:00,None,2021-07-21 16:09:54.130000+00:00,NaT,None,NaN,None,14132876,68471992,2,0,None,None
2,68472975,None,<p>I believe <code>Identifiable</code> is a pr...,None,None,1,NaT,2021-07-21 16:09:56.317000+00:00,None,2021-07-21 16:09:56.317000+00:00,NaT,None,NaN,None,16487083,68472232,2,0,None,None
3,68472976,None,<p>Funny enough I had the same problem with th...,None,None,0,NaT,2021-07-21 16:10:06.223000+00:00,None,2021-07-21 16:10:06.223000+00:00,NaT,None,NaN,None,9497800,64728953,2,0,None,None
4,68472989,None,<p>You either need to set a <code>height</code...,None,None,0,NaT,2021-07-21 16:10:45.250000+00:00,None,2021-07-21 16:10:45.250000+00:00,NaT,None,NaN,None,6035993,68472903,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [5]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,36723113,webservice handling protobuf,<p>I'm trying to make a webservice work with p...,36739655.0,1,0,NaT,2016-04-19 15:37:01.167000+00:00,NaN,2016-04-20 09:36:16.880000+00:00,NaT,None,NaN,None,821464,None,1,0,java|spring|web-services|protocol-buffers,256
1,36731285,Get sheets belonging to a specific group,<p>How do I tell if a sheet belongs to a certa...,36738605.0,1,0,NaT,2016-04-19 23:35:10.823000+00:00,NaN,2016-04-20 08:55:31.697000+00:00,NaT,None,NaN,None,1203098,None,1,1,smartsheet-api|smartsheet-c#-sdk-v2,256
2,36732239,Deploying OSGi bundle on Oracle 12c throws NPE,"<p>On Weblogic 12.2.1, I am trying to deploy a...",NaN,1,0,NaT,2016-04-20 01:24:19.777000+00:00,NaN,2016-04-20 07:58:05.090000+00:00,2016-04-20 03:30:31.183000+00:00,None,1247961.0,None,1247961,None,1,1,osgi|weblogic12c|apache-felix,256
3,36792175,Starscream's socket.isConnected is always retu...,<p>I'm developing an iOS application using Swi...,NaN,1,0,NaT,2016-04-22 11:07:16.993000+00:00,NaN,2017-08-02 10:58:43.600000+00:00,NaT,None,NaN,None,3114316,None,1,1,ios|swift|sockets,256
4,36804673,Connection Autodesk Motionbuilder with iOS app...,<p>I am trying to write a C++ api to connect A...,NaN,1,0,NaT,2016-04-22 23:13:03.953000+00:00,NaN,2016-04-28 20:44:34.633000+00:00,NaT,None,NaN,None,6242858,None,1,0,ios|osc|motionbuilder,256


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [6]:

questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query,job_config=safe_config)

questions_results = questions_query_job.to_dataframe() 


print(questions_results.head())


q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


         id                                              title  owner_user_id
0  64487437  Strange repeated field error when uploading to...       188963.0
1  64310251  Is it possible to make a custom metric about a...     14351627.0
2  64472919  Why does Azure Data Factory Integration Runtim...      3254527.0
3  64352958        Mixed Billing Methods In a Bigquery Project     11692390.0
4  64134119  Google BigQuery Superimpose Two Tables with Id...      7326954.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.



It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [7]:

answers_query = """SELECT ans.id, ans.body, ans.owner_user_id
                   FROM `bigquery-public-data.stackoverflow.posts_answers` ans
                   INNER JOIN `bigquery-public-data.stackoverflow.posts_questions` q
                   ON ans.parent_id=q.id
                   WHERE q.tags LIKE '%bigquery%'
                   """


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=27*10**10)
answers_query_job = client.query(answers_query,job_config=safe_config) 


answers_results = answers_query_job.to_dataframe() 


print(answers_results.head())

# Check your answer
q_4.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


         id                                               body  owner_user_id
0  64851746  <p>I figured it out using a sub query, hopeful...       789938.0
1  64852758  <p>It looks like in my particular case, the is...      1362318.0
2  64859159  <p>When you attempt to test a scheduled query ...     12942591.0
3  64864200  <p>If I understand correctly, your cluster is ...      9457843.0
4  64872364  <p>According to my experience with GCP, both C...     14565295.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [8]:
# Your code here
bigquery_experts_query = """
                   WITH bigqueryans AS 
                   (SELECT ans.id, ans.body, ans.owner_user_id
                   FROM `bigquery-public-data.stackoverflow.posts_answers` ans
                   INNER JOIN `bigquery-public-data.stackoverflow.posts_questions` q
                   ON ans.parent_id=q.id
                   WHERE q.tags LIKE '%bigquery%')
                   
                   SELECT owner_user_id as user_id, COUNT(1) as number_of_answers
                   FROM bigqueryans
                   GROUP BY owner_user_id
                   ORDER BY number_of_answers
                   """


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query,job_config=safe_config) 


bigquery_experts_results = bigquery_experts_query_job.to_dataframe()# Your code goes here


print(bigquery_experts_results.head())


q_5.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


      user_id  number_of_answers
0   4461381.0                  1
1   9282069.0                  1
2  10930378.0                  1
3   4058781.0                  1
4   4165839.0                  1


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [9]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%{topic}%'
               GROUP BY a.owner_user_id
               """
               
    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)
    
    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results

```

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*